# multiscale-galactose example
Example simulations for the multiscale model

In [1]:
# Testing of load times for model
from __future__ import print_function, division
import roadrunner
print(roadrunner.__version__)
import time

def time_it(func, *args, **kwargs):
    """ Time the call to the function. """
    time_start = time.time()
    res = func(*args, **kwargs)
    print('Time: {}'.format(time.time() - time_start))
    return res

def hash_for_file(filepath, hash_type='MD5', blocksize=65536):
    """ Calculate the md5_hash for a file.

        Calculating a hash for a file is always useful when you need to check if two files
        are identical, or to make sure that the contents of a file were not changed, and to
        check the integrity of a file when it is transmitted over a network.
        he most used algorithms to hash a file are MD5 and SHA-1. They are used because they
        are fast and they provide a good way to identify different files.
        [http://www.pythoncentral.io/hashing-files-with-python/]
    """
    import hashlib

    hasher = None
    if hash_type == 'MD5':
        hasher = hashlib.md5()
    elif hash_type == 'SHA1':
        hasher == hashlib.sha1()
    with open(filepath, 'rb') as f:
        buf = f.read(blocksize)
        while len(buf) > 0:
            hasher.update(buf)
            buf = f.read(blocksize)
    return hasher.hexdigest()

test_sbml = '../models/galactose/Galactose_v128_Nc20_dilution.xml'

print('*** First loading ***')
# r = rt.load_model(test_sbml)
r = time_it(roadrunner.RoadRunner, test_sbml)

print('*** Reloading ***')
r = time_it(roadrunner.RoadRunner, test_sbml)

print('*** Hash calculation MD5 ***')
time_it(hash_for_file, test_sbml)

1.4.2; Compiler: gcc 4.8.4, C++ version: 199711; JIT Compiler: LLVM-3.4; Date: Feb  1 2016, 15:48:52; LibSBML Version: 5.12.0
*** First loading ***
Time: 31.513780117
*** Reloading ***
Time: 2.22731184959
*** Hash calculation MD5 ***
Time: 0.00426912307739


'c637410a85d09370be99db1cd0bd44ac'

In [ ]:
import roadrunner
from roadrunner import SelectionRecord

# import multiscale.examples.testdata as testdata
print(test_sbml)

# 31s, reload 2.7s (core), 
r = time_it(roadrunner.RoadRunner, test_sbml)
integrator = r.getIntegrator()

# set integrator settings
rel_tol = 1E-6
abs_tol = 1E-6 * min(r.model.getCompartmentVolumes())
integrator.setValue('absolute_tolerance', abs_tol)
integrator.setValue('relative_tolerance', rel_tol)
tend = 1000

# working
print('*** stiff=True, variable_step_size=False ***')
integrator.setValue('stiff', True)
integrator.setValue('variable_step_size', False)
print(integrator)
r.reset(SelectionRecord.ALL)
time_it(r.simulate, start=0, end=tend, steps=1000)

# working
print('*** stiff=True, variable_step_size=True ***')
integrator.setValue('stiff', True)
integrator.setValue('variable_step_size', True)
print(integrator)
r.reset(SelectionRecord.ALL)
time_it(r.simulate, start=0, end=tend, steps=1000)

# uses up all the available memory & brings the server down!
# probably maximum_num_steps: 20000 not tested
print('*** stiff=False, variable_step_size=True ***')
integrator.setValue('stiff', False)
integrator.setValue('variable_step_size', True)
print(integrator)
r.reset(SelectionRecord.ALL)
time_it(r.simulate, start=0, end=tend);


"""
# handling correctly via RuntimeError or running forever 
print('*** stiff=False, variable_step_size=False ***')
integrator.setValue('stiff', False)
integrator.setValue('variable_step_size', False)
print(integrator)
r.reset(SelectionRecord.ALL)
time_it(r.simulate, start=0, end=tend, steps=1000)
"""

../models/galactose/Galactose_v128_Nc20_dilution.xml
Time: 2.63174796104
*** stiff=True, variable_step_size=False ***
< roadrunner.Integrator() >
  settings:
      relative_tolerance: 0.000001
      absolute_tolerance: 1.5205308443374601e-21
                   stiff: true
       maximum_bdf_order: 5
     maximum_adams_order: 12
       maximum_num_steps: 20000
       maximum_time_step: 0
       minimum_time_step: 0
       initial_time_step: 0
          multiple_steps: false
      variable_step_size: false

Time: 1.80528187752
*** stiff=True, variable_step_size=True ***
< roadrunner.Integrator() >
  settings:
      relative_tolerance: 0.000001
      absolute_tolerance: 1.5205308443374601e-21
                   stiff: true
       maximum_bdf_order: 5
     maximum_adams_order: 12
       maximum_num_steps: 20000
       maximum_time_step: 0
       minimum_time_step: 0
       initial_time_step: 0
          multiple_steps: false
      variable_step_size: true

Time: 1.7042760849

In [ ]:
import tellurium as te
model = '''
    model pathway()
    S1 -> S2; k1*S1 - k2*S2 # Reversible term added here

    # Initialize values
    S1 = 5; S2 = 0;
    k1 = 0.1;  k2 = 0.05;
end
'''
r = te.loada(model)
r.simulate(0,10)

# 
from roadrunner import SelectionRecord
integrator = r.getIntegrator()
integrator.setValue('stiff', True)
integrator.setValue('variable_step_size', True)
print(integrator)
r.reset()
print(integrator)
r.reset(SelectionRecord.ALL)
print(integrator)